# Dates Extraction

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [3]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [25]:
def date_sorter():
    
    # Your code here
    
    ### type 1 (190): 04/20/2009; 04/20/09; 4/20/09; 4/3/09; 4-13-82
    dates1 = df.str.extract(r'(\d?\d)(?:-|/)(\d?\d)(?:-|/)(\d{2,4})')
    dates1 = dates1.dropna()
    dates1[0] = dates1[0].apply(lambda s:"0"+s if type(s)==str and len(s)==1 else s)
    dates1[1] = dates1[1].apply(lambda s:"0"+s if type(s)==str and len(s)==1 else s)
    dates1[2] = dates1[2].apply(lambda s:"19"+s if type(s)==str and len(s)==2 else s)
    #print(len(dates1)-len(dates1.dropna()))
    
    
    ### type 2 (34): Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009
    month_mapping = {"jan":"01","feb":"02","mar":"03","apr":"04","may":"05","jun":"06",
                    "jul":"07","aug":"08","sep":"09","oct":"10","nov":"11","dec":"12"}

    dates2 = df.str.extract(r'([A-z]+.)(?:-|\s)(\d\d?)(?:-|\s|,\s)(\d{4})')
    dates2 = dates2.dropna()
    #dates cannot be greater than 31
    dates2 = dates2[dates2[1].astype(int) <= 31]
    #month must be in the standard format
    dates2[0] = dates2[0].str[:3].str.lower()
    dates2 = dates2[dates2[0].isin(month_mapping)]
    dates2[0] = dates2[0].map(month_mapping)
    
    
    ### type 3 (69): 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    dates3 = df.str.extract(r'(\d\d?)(?:-|\s)([A-z]+.)(?:-|\s|,\s)(\d{2,4})')
    dates3 = dates3.dropna()
    #dates cannot be greater than 31
    dates3 = dates3[dates3[0].astype(int) <= 31]
    #month must be in the standard format
    dates3[1] = dates3[1].str[:3].str.lower()
    dates3 = dates3[dates3[1].isin(month_mapping)]
    dates3[1] = dates3[1].map(month_mapping)
    dates3 = dates3[[1,0,2]]
    dates3.columns = [0,1,2]
    
    
    ### type 4 (0): Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
    dates4 = df.str.extract(r'([A-z]+.)(?:-|\s)(\d\d?\w\w)(?:-|\s|,\s)(\d{4})')
    dates4 = dates4.dropna()
    #dates cannot be greater than 31
    dates4 = dates4[dates4[1].astype(int) <= 31]
    
    ### type 5 (157): Feb 2009; Sep 2009; Oct, 2010
    dates5 = df.str.extract(r'([A-z]+,?)(?:-|\s)(\d{4})')
    dates5 = dates5.dropna()
    #month must be in the standard format
    dates5[0] = dates5[0].str.lower()
    
    dates5s = []
    
    for s in month_mapping:
        dates5_ = dates5[dates5[0].str.contains(s)]
        dates5_[0] = [s] * len(dates5_[0])
        dates5s.append(dates5_)
        
    dates5 = pd.concat(dates5s)
    
    dates5[0] = dates5[0].map(month_mapping)
    dates5[2] = ["01"]*len(dates5)
    dates5 = dates5[[0,2,1]]
    dates5.columns = [0,1,2]
    
    ### type 6 (): 6/2008; 12/2009
    dates6 = df.str.extract(r'(\d?\d)/(\d{4})')
    dates6 = dates6.dropna()
    #month must be in the standard format
    dates6 = dates6[dates6[0].astype(int) <= 12]
    dates6[0] = dates6[0].apply(lambda s:"0"+s if len(s) == 1 else s)
    dates6[1] = dates6[1].apply(lambda s:"19"+s if len(s) == 2 else s)
    dates6[2] = ["01"] * len(dates6)
    dates6 = dates6[[0,2,1]]
    dates6.columns = [0,1,2]
    
    ### type 7 (): 2009; 2010
    dates72 = df.str.extract(r'(\d{4})')
    dates72 = dates72.dropna()
    dates72 = dates72[dates72.astype(int)<2019]
    dates7 = pd.DataFrame()
    dates7[2] = dates72
    dates7[1] = ["01"] * len(dates7)
    dates7[0] = ["01"] * len(dates7)
    
    res_dict = dates7.to_dict()
    for d in [dates6,dates5,dates4,dates3,dates2,dates1]:
        df_dict = d.to_dict()
        for idx in [0,1,2]:
            res_dict[idx].update(df_dict[idx])
    
    df_res = pd.DataFrame(res_dict).astype(int)
    df_res["text"] = df
    #df_res = df_res.reset_index()
    #df_res = df_res.sort_values(axis=0,by=["index"],ascending=False)
    df_res = df_res.sort_values(by=[2,0,1],axis=0)
    #df_res.to_csv("submission.csv")
    
    return pd.Series(df_res.index) # Your answer here


#date_sorter()